# 结论：
1、dftest的user_id中，只有2个没有在dfoff中出现过【2495873，1286474】    
2、dftest有约10万个UM_id，但是其中9万个在nf2中没有记录  
3、同上，valid有约21万个UM_id,但是其中18.5万个id在nf2中没有记录  

In [1]:
# 目前只有两个特征比较有效，希望从这两个特征入手，看看核销的人有什么特征

In [14]:
import sys
sys.path.append('../app/')
import fileio
import featureprocess
import preprocess
import matplotlib.pyplot as plt

In [7]:
dfoff,dftest,dfon = fileio.read_all()

In [12]:
dfoff,dfall,train,valid = preprocess.preprocess(dfoff)

In [13]:
dfoff = featureprocess.process_features_main(dfoff)
dfall = featureprocess.process_features_main(dfall)

train = featureprocess.process_features_main(train)
valid = featureprocess.process_features_main(valid)
dftest = featureprocess.process_features_main(dftest)

In [17]:
# 各个特征，柱状图
valid.weekday

0         6
1         1
2         1
3         1
4         4
         ..
252581    1
252582    2
252583    1
252584    1
252585    1
Name: weekday, Length: 252586, dtype: int64